In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from gelos.gelosdataset import GELOSDataSet
from gelos.gelosdatamodule import GELOSDataModule
import yaml
from gelos import config
from lightning.pytorch import Trainer


2025-11-26 02:49:41.886 | INFO     | gelos.config:<module>:11 - PROJ_ROOT path is: /app
/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations
/opt/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a sing

In [3]:
yaml_config_directory = config.PROJ_ROOT / 'gelos' / 'configs'
yaml_name = 'prithvi_eo_300m_embedding_generation.yaml'
with open(yaml_config_directory / yaml_name, "r") as f:
    yaml_config = yaml.safe_load(f)
print(yaml.dump(yaml_config))

data:
  class_path: gelos.gelosdatamodule.GELOSDataModule
  init_args:
    bands:
      S2L2A:
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - NIR_BROAD
      - NIR_NARROW
    batch_size: 1
    num_workers: 0
model:
  class_path: terratorch.tasks.EmbeddingGenerationTask
  init_args:
    embed_file_key: filename
    embedding_pooling: vit_cls
    has_cls: true
    model: prithvi_eo_v2_300
    model_args:
      backbone: prithvi_eo_v2_300
      backbone_bands:
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - NIR_BROAD
      - NIR_NARROW
      backbone_pretrained: true
    output_format: parquet
seed_everything: 0
trainer:
  accelerator: auto
  callbacks: []
  devices: auto
  max_epochs: 0
  num_nodes: 1
  strategy: auto



In [4]:
model_name = yaml_config['model']['init_args']['model']
output_dir = config.INTERIM_DATA_DIR / config.DATA_VERSION / model_name
output_dir.mkdir(exist_ok=True)

data_root = config.RAW_DATA_DIR / config.DATA_VERSION

# add variables to yaml config so it can be passed to classes
yaml_config['data']['init_args']['data_root'] = data_root
yaml_config['model']['init_args']['output_dir'] = output_dir

In [5]:
gelos_datamodule = GELOSDataModule(**yaml_config['data']['init_args'])

In [7]:
from terratorch.tasks import EmbeddingGenerationTask

In [8]:
class LenientEmbeddingGenerationTask(EmbeddingGenerationTask):
    
    def check_file_ids(self, file_ids, x):
        return

In [9]:
task = LenientEmbeddingGenerationTask(**yaml_config['model']['init_args'])

INFO:terratorch.models.backbones.prithvi_vit:model_bands not passed. Assuming bands are ordered in the same way as [<HLSBands.BLUE: 'BLUE'>, <HLSBands.GREEN: 'GREEN'>, <HLSBands.RED: 'RED'>, <HLSBands.NIR_NARROW: 'NIR_NARROW'>, <HLSBands.SWIR_1: 'SWIR_1'>, <HLSBands.SWIR_2: 'SWIR_2'>].Pretrained patch_embed layer may be misaligned with current bands


In [10]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
trainer = Trainer(accelerator=device, devices=1)


INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [12]:
%pdb

Automatic pdb calling has been turned ON


In [13]:
trainer.predict(model=task, datamodule=gelos_datamodule)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/opt/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 0/77547 [00:33<?, ?it/s]


RuntimeError: Model inference failed: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

> /opt/venv/lib/python3.11/site-packages/terratorch/tasks/embedding_generation.py(142)get_embeddings()
    140             outputs = self.model(input)
    141         except Exception as e:
--> 142             raise RuntimeError(f"Model inference failed: {e}")
    143 
    144         if not isinstance(outputs, list):

self = LenientEmbeddingGenerationTask(
  (model): PrithviViT(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(6, 1024, kernel_size=(1, 16, 16), stride=(1, 16, 16))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-23): 24 x Block(
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (norm): Identity()
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, in